## Netejar dataset

In [ ]:
import re
import torch
import pandas as pd

In [ ]:
df = pd.read_csv('data/chordomicon_clean.csv')
# Configuració
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

Eliminar files que tenen la columna "main_genre" buida

In [ ]:
# Contar valors buits (NaN) en la columna 'main_genre'
print("Files amb 'main_genre' buit abans de netejar:", df['main_genre'].isna().sum())

df_clean = df.dropna(subset=['main_genre'])
print("Files després de netejar:", len(df_clean))
print("Files amb 'main_genre' després:", df_clean['main_genre'].isna().sum())

drop_columns = ["id", "s_artist", "release_date", "genres", "decade", "rock_genre", "parts"]

df_clean = df_clean.drop(columns=drop_columns, axis=1)
print("Columnes finals:", df_clean.columns.tolist())


Normalitzar els acords

In [ ]:
# Mapeig d'enarmònics per normalitzar
enharmonic_map = {
    'Cb': 'B', 'Db': 'C#', 'Eb': 'D#', 'Fb': 'E', 'Gb': 'F#', 'Ab': 'G#', 'Bb': 'A#',
    'E#': 'F', 'B#': 'C'
}

# Funció per simplificar i normalitzar un acord
def simplify_chord(chord):
    if chord.startswith('<') and chord.endswith('>'):
        # Elimina el número del tag: <verse_1> → <verse>
        return re.sub(r'_\d+', '', chord)

    # Coincideix acords com Cm, C, Cmaj7, etc.
    match = re.match(r'^([A-G][b#]?)(m)?', chord)
    if not match:
        return chord  # Retorna el valor tal qual si no coincideix

    root, minor = match.groups()
    root = enharmonic_map.get(root, root)  # Normalitza enarmònics

    return root + ('m' if minor else '') # Si l'acord és menor, afegeix 'm', sinó retorna només la nota


# Aplica la normalització i simplificació
df_clean['chords'] = df_clean['chords'].apply(
    lambda seq: ' '.join([simplify_chord(ch) for ch in seq.split()])
)

In [ ]:
# Verificar resultats
print("\nExemple d'acords normalitzats:")
print(df_clean['chords'].head())

Guardar dataset

In [ ]:
df.to_csv('data/chordomicon_clean.csv', index=False)

Resum de gèneres musicals del dataset

In [ ]:
genre_counts = df['main_genre'].value_counts()

print("Gèneres totals:\n", genre_counts)